In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import torch

if torch.cuda.is_available():
    device = "cuda"
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ Using GPU: {gpu_name}")
else:
    device = "cpu"
    print("⚠️ CUDA not available. Using CPU instead.")

print(f"🖥️ Device set to: {device}")

✅ Using GPU: NVIDIA GeForce RTX 4080 SUPER
🖥️ Device set to: cuda


In [3]:
# ==============================
# Imports
# ==============================
import pandas as pd
import torch
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [4]:


# ==============================
# Set random seeds for reproducibility
# ==============================
# 💡 Added this block to ensure consistent results across runs
seed = 20
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:

# Step 1: Training data 4 shot inltk news urdu
train_dataset = [
    # entertainment 4shots
    InputExample(guid=0, text_a="ایکتا کپور نے بنائی ہندوستان کی سب سے بولڈ ویب سیریز ایکس ایکس ایکس ، یہاں دیکھیں ٹریلر", label=0),
    InputExample(guid=1, text_a="فوٹو شوٹ کرانا پڑا سارہ علی خان کو بےحد مہنگا، سوشل میڈیا صارفین نے جم کر کیا ٹرول", label=0),
    InputExample(guid=2, text_a="عامر خان کی وجہ سے گھنٹوں باتھ روم میں بیٹھ کر روئی یہ اداکارہ، سلمان خان نے بچایا", label=0),
    InputExample(guid=3, text_a="ایشا امبانی پری ویڈنگ : جب اپنے ' ہیرو ' سے ملنے کیلئے ودیا بلن نے کی اسمرتی ایرانی سے درخواست", label=0),


    # Cricket 4 shot
    InputExample(guid=4, text_a="آسٹریلیا کے خلاف پہلے ٹیسٹ کیلئے ٹیم انڈیا کا اعلان ، امیش اور جڈیجہ باہر ، مگر اس بڑے کھلاڑی کو ملی جگہ", label=1),
    InputExample(guid=5, text_a="پاکستانی کوچ مکی آرتھرنے بابراعظم کی کارکردگی اورمستقبل سے متعلق کہی یہ بات", label=1),
    InputExample(guid=6, text_a="پاکستانی ٹیم کو لگا بڑا جھٹکا ، نیوزی لینڈ کے خلاف سیریز سے بھی باہر ہوسکتا ہے یہ بڑا کھلاڑی", label=1),
    InputExample(guid=7, text_a="گیند بازی کے بعد بلے بازی میں بھی راشد خان کا دھماکہ ، تین گیندوں میں تین چھکے لگا کر ٹیم کو دلائی جیت", label=1),

    # crime 4shot
    InputExample(guid=8, text_a="دادری سانحہ: مشتعل بھیڑ اخلاق کے پورے خاندان کو زندہ جلا دینا چاہتی تھی", label=2),
    InputExample(guid=9, text_a="چھیڑ چھاڑ کی مخالفت کی تو گاوں والوں نے اسکول میں گھس کر کی پٹائی ، 34 طالبات زخمی", label=2),
    InputExample(guid=10, text_a="اب یوپی کے دیوریا میں مظفرپور جیسے سیکس ریکٹ کا پردہ فاش، 24 خواتین آزاد کرائی گئیں", label=2),
    InputExample(guid=11, text_a="جموں و کشمیر : گھر لوٹ رہی خاتون کو گھسیٹ کر لے گیا جنگل ، آبروریزی کی ، بتانے پر سنگین نتائج کی دی دھمکی", label=2),

]


In [6]:
# ==============================
# Define Classes
# ==============================
classes = ["entertainment", "cricket", "crime"]
label_map = {"entertainment": 0, "cricket": 1, "crime": 2}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',
    tokenizer=tokenizer,
)


# automatic verbalizer
verbalizer = AutomaticVerbalizer(
    tokenizer=tokenizer,
    classes=classes,
    num_candidates=1000,
    label_word_num_per_class=5,
)

In [8]:



# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # Even though shuffled, reproducibility is preserved by seeding
)

# ==============================
# Fine-Tuning the Prompt Model
# ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(5):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Downloads\INLTK_urdu\test.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['headline'], label=label_map[row['label']])
    for i, row in df.iterrows()
]


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))


tokenizing: 12it [00:00, 1496.63it/s]


Epoch 1 Loss: 5.3057
Epoch 2 Loss: 3.4287
Epoch 3 Loss: 3.8526
Epoch 4 Loss: 3.6253
Epoch 5 Loss: 3.7148


tokenizing: 2239it [00:01, 1844.68it/s]



📊 Fine-tuned Classification Report:
               precision    recall  f1-score   support

entertainment     0.5566    0.3252    0.4105      1270
      cricket     0.3279    0.3443    0.3359       700
        crime     0.1155    0.3271    0.1707       269

     accuracy                         0.3314      2239
    macro avg     0.3333    0.3322    0.3057      2239
 weighted avg     0.4321    0.3314    0.3584      2239

